### Networks

<b>O que são Networks no Docker?</b>

- Uma forma de gerenciar a conexão do Docker com outras plataformas ou até mesmo entre containers;

- As redes ou networks são criadas separadas do containers, como os volumes;

- Além disso existem alguns drivers de rede, que veremos em seguida;

- Uma rede deixa muito simples a comunicação entre containers;


<b>Tipos de rede (drivers)</b>

- ```Bridge```: o mais comum e default do Docker, utilizado quando containers precisam se conectar (na maioria das vezes optamos por este driver);

- ```host```: permite a conexão entre um container a máquina que está hosteando o Docker;

- ```macvlan```: permite a conexão a um container por um MAC address;

- ```none```: remove todas conexões de rede de um container;

- ```plugins```: permite extensões de terceiros para criar outras redes;

<b>Tipos de conexão</b>

- Os containers costumam ter três principais tipos de comunicação:

- ```Externa```: conexão com uma API de um servidor remoto;

- ```Com o host```: comunicação com a máquina que está executando o Docker;

- ```Entre containers```: comunicação que utiliza o driver bridge e permite a comunicação entre dois ou mais containers;

<b>Listando redes</b>

- Podemos verificar todas as redes do nosso ambiente com: ```docker network ls;```

- Algumas redes já estão criadas, estas fazem parte da configuração inicial do docker; 

In [1]:
! docker network ls

NETWORK ID     NAME                          DRIVER    SCOPE
db83cc9843e9   bridge                        bridge    local
43002a0cebb8   emsoutlierpredictor_default   bridge    local
1cfbd8b25862   host                          host      local
2ae37ceb6410   lifetest-api_default          bridge    local
72f85eb5c733   none                          null      local


<b>Criando rede</b>

- Para criar uma rede vamos utilizar o comando ```docker network create <nome>```;

- Esta rede será do tipo bridge, que é o mais utilizado;

- Podemos criar diversas redes

In [2]:
! docker network create rede-manual

1dd21d8919f9e4a7980e2a27188fd91e000f9a96a406134a52e2943e7a7e84fc


In [3]:
! docker network ls

NETWORK ID     NAME                          DRIVER    SCOPE
db83cc9843e9   bridge                        bridge    local
43002a0cebb8   emsoutlierpredictor_default   bridge    local
1cfbd8b25862   host                          host      local
2ae37ceb6410   lifetest-api_default          bridge    local
72f85eb5c733   none                          null      local
1dd21d8919f9   rede-manual                   bridge    local


In [4]:
! docker network create -d macvlan rede-macvlan

96159929a577d68c06f90bc073cf81dfd5cf9e095878cfb2b76f1c6c87640900


In [5]:
! docker network ls

NETWORK ID     NAME                          DRIVER    SCOPE
db83cc9843e9   bridge                        bridge    local
43002a0cebb8   emsoutlierpredictor_default   bridge    local
1cfbd8b25862   host                          host      local
2ae37ceb6410   lifetest-api_default          bridge    local
72f85eb5c733   none                          null      local
96159929a577   rede-macvlan                  macvlan   local
1dd21d8919f9   rede-manual                   bridge    local


<b>Removendo redes</b>

- Podemos remover redes de forma simples também: ```docker network rm <nome>```;

- Assim a rede não estará mais disponível para utilizarmos;

- Devemos tomar cuidado com containers já conectados;

In [6]:
! docker network rm rede-macvlan

rede-macvlan


In [7]:
! docker network ls

NETWORK ID     NAME                          DRIVER    SCOPE
db83cc9843e9   bridge                        bridge    local
43002a0cebb8   emsoutlierpredictor_default   bridge    local
1cfbd8b25862   host                          host      local
2ae37ceb6410   lifetest-api_default          bridge    local
72f85eb5c733   none                          null      local
1dd21d8919f9   rede-manual                   bridge    local


<b>Removendo redes em massa</b>

- Podemos remover redes de forma simples também: ```docker network prune```;

- Assim todas as redes não utilizadas no momento serão removidas;

- Receberemos uma mensagem de confirmação do Docker antes da ação ser executada;

In [9]:
! docker network prune -f

Deleted Networks:
lifetest-api_default
rede-manual



<b>Instalação do Postman</b>

- Vamos criar uma API para testar a conexão entre containers;

- Para isso vamos utilizar o software Postman, que é o mais utilizado do mercado para desenvolvimento de APIs;

- Link: https://www.postman.com/

<b>Conexão externa</b>

- Os containers podem se conectar livremente ao mundo externo;

- Um caso seria: uma API de código aberto;

- Podemos acessá-la livremente e utilizar seus dados;

- Vamos testar!

<b>Criando o arquivo Docker</b>

```Dockerfile
FROM python:3

RUN apt-get update -y && \
  apt-get install -y python3-pip python3-dev

WORKDIR /app

RUN pip install Flask
RUN pip install requests

COPY . .

EXPOSE 5000

CMD ["python", "./app.py"]

```

<b>Criando o arquivo Python</b>

```python
import flask
from flask import request, json, jsonify
import requests

app = flask.Flask(__name__)
app.config["DEBUG"] = True

@app.route("/", methods=["GET"])
def index():
  data = requests.get('https://randomuser.me/api')
  return data.json()

if __name__ == "__main__":
  app.run(host="0.0.0.0", debug=True, port="5000")

```

In [ ]:
! docker build -t bruiglesias/flask-image .

In [10]:
! docker run -d -p 5000:5000 --rm --name flask-container bruiglesias/flask-image

826ef2a25bfd0c4842a66a124697e733d493888e1aac27935a75db9aaad3d510


In [11]:
! docker ps

CONTAINER ID   IMAGE                                        COMMAND                  CREATED          STATUS                 PORTS                                       NAMES
826ef2a25bfd   bruiglesias/flask-image                      "python ./app.py"        25 seconds ago   Up 24 seconds          0.0.0.0:5000->5000/tcp, :::5000->5000/tcp   flask-container
49edf471047e   extending_airflow:latest                     "/usr/bin/dumb-init …"   3 hours ago      Up 3 hours (healthy)   0.0.0.0:8080->8080/tcp, :::8080->8080/tcp   emsoutlierpredictor-airflow-webserver-1
9c15868999a6   extending_airflow:latest                     "/usr/bin/dumb-init …"   3 hours ago      Up 3 hours             8080/tcp                                    emsoutlierpredictor-airflow-scheduler-1
2bb8a1667a31   postgres:13                                  "docker-entrypoint.s…"   3 hours ago      Up 3 hours (healthy)   0.0.0.0:5432->5432/tcp, :::5432->5432/tcp   emsoutlierpredictor-postgres-1
62f5afc30660   mcr.mic

<b>Conexão com o host</b>

- Podemos também conectar um container com o host do Docker;

- Host é a máquina que está executando o Docker;

- Como ip de host utilizamos: host.docker.internal

- No caso pode ser a nossa mesmo! =) 

<b>Criando o arquivo Docker</b>

```Dockerfile
FROM python:3

RUN apt-get update -y && \
  apt-get install -y python3-pip python3-dev

WORKDIR /app

RUN pip install Flask requests flask_mysqldb

COPY . .

EXPOSE 5000

CMD ["python", "./app.py"]
```

<b>Criando o arquivo Python</b>

```python
import flask
from flask import request, json, jsonify
import requests
import flask_mysqldb
from flask_mysqldb import MySQL

app = flask.Flask(__name__)
app.config["DEBUG"] = True

app.config['MYSQL_HOST'] = 'host.docker.internal'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''
app.config['MYSQL_DB'] = 'flaskdb'

mysql = MySQL(app)

@app.route("/", methods=["GET"])
def index():
  data = requests.get('https://randomuser.me/api')
  return data.json()

@app.route("/inserthost", methods=['POST'])
def inserthost():
  data = requests.get('https://randomuser.me/api').json()
  username = data['results'][0]['name']['first']

  cur = mysql.connection.cursor()
  cur.execute("""INSERT INTO users(name) VALUES(%s)""", (username,))
  mysql.connection.commit()
  cur.close()

  return username

if __name__ == "__main__":
  app.run(host="0.0.0.0", debug=True, port="5000")

```

In [ ]:
! docker build -t bruiglesias/flask-image .

In [ ]:
! docker run -d -p 5000:5000 --rm --name flask-container bruiglesias/flask-image

<b>Conexão entre containers</b>

- Podemos também estabelecer uma conexão entre containers;

- Duas imagens distintas rodando em containers separados que precisam se conectar para inserir um dado no banco, por exemplo;

- Vamos precisar de uma rede bridge, para fazer esta conexão;

- Agora nosso container de flask vai inserir dados em um MySQL que roda pelo Docker também;

<b>Criando o arquivo Docker</b>

```Dockerfile
FROM mysql:5.7

COPY schema.sql /docker-entrypoint-initdb.d/

EXPOSE 3306

VOLUME ["/backup/"]
```

<b>Criando o arquivo SQL</b>

```sql
CREATE DATABASE flaskdb;

USE flaskdb;

CREATE TABLE `flaskdb`.`users` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `name` VARCHAR(255),
  PRIMARY KEY (ID));

```

<b>Criando o arquivo Python</b>

```python
import flask
from flask import request, json, jsonify
import requests
import flask_mysqldb
from flask_mysqldb import MySQL

app = flask.Flask(__name__)
app.config["DEBUG"] = True

app.config['MYSQL_HOST'] = 'mysql-container'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''
app.config['MYSQL_DB'] = 'flaskdb'

mysql = MySQL(app)

@app.route("/", methods=["GET"])
def index():
  data = requests.get('https://randomuser.me/api')
  return data.json()

@app.route("/inserthost", methods=['POST'])
def inserthost():
  data = requests.get('https://randomuser.me/api').json()
  username = data['results'][0]['name']['first']

  cur = mysql.connection.cursor()
  cur.execute("""INSERT INTO users(name) VALUES(%s)""", (username,))
  mysql.connection.commit()
  cur.close()

  return username

if __name__ == "__main__":
  app.run(host="0.0.0.0", debug=True, port="5000")
  ```

! docker build -t bruiglesias/mysql-image .

In [ ]:
! docker run -d -p 3306:3306 --name mysql-container --rm --network flask-network -e MYSQL_ALLOW_EMPTY_PASSWORD=True bruiglesias/mysql-image

In [ ]:
! docker build -t bruiglesias/flask-image .

In [ ]:
! docker run -d -p 5000:5000 --name flask-container --rm --network flask-network bruiglesias/flask-image

<b>Conectar container</b>

- Podemos conectar um container a uma rede;

- Vamos utilizar o comando ```docker network connect <rede> <container>```

- Após o comando o container estará dentro da rede! 

In [ ]:
! docker network connect flask-network flask-container

<b>Desconectar container</b>

- Podemos desconectar um container a uma rede também;

- Vamos utilizar o comando ```docker network disconnect <rede> <container>```

- Após o comando o container estará fora da rede!

In [ ]:
! docker network disconnect flask-network flask-container


<b>Inspecionando redes</b>

- Podemos analisar os detalhes de uma rede com o comando: ```docker network inspect <nome>```

- Vamos receber informações como: data de criação, driver, nome e muito mais!

In [ ]:
! docker network inspect flask-network